In [1]:
import numpy as np
import pandas as pd

In [8]:
data = np.load("train.npy")

In [ ]:
X = []
Y = []
i = 0
for elem in data:
    if i % 1000 == 0:
        print(i)
    X.append(tf.resize(elem[0], output_shape=(40,40)))
    Y.append(elem[1])
    i += 1

In [12]:
np.save('X', X)

In [13]:
np.save('Y', Y)

In [29]:
X = np.array(X)
Y = np.array(Y)

In [2]:
X = np.load("X.npy")
Y = np.load("Y.npy")

In [3]:
X.resize((166708,40,40,1))

In [4]:
labels = np.unique(Y)
labels = labels.tolist()

In [5]:
for i in range(len(Y)):
    Y[i] = labels.index(Y[i])

In [28]:
X = X.astype(np.float16)
Y = Y.astype(np.int16)

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K

batch_size = 128
nb_classes = 500
nb_epoch = 10

# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (40,40,1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()



model.add(Convolution2D(80, kernel_size[0] + 2, kernel_size[1] + 2,
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(1,1)))



model.add(Convolution2D(160, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))
          
model.add(BatchNormalization())


model.add(Convolution2D(240, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))


model.add(Convolution2D(320, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))


model.add(Convolution2D(350, kernel_size[0], kernel_size[3]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))


model.add(Flatten())

model.add(BatchNormalization())

model.add(Dense(400))
model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=2, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


In [10]:
from keras.models import load_model


model = load_model("./Models Architecture/4 conv with maxpool and batchnorm V2")

In [11]:
from keras.utils import np_utils

batch_size = 128
nb_classes = 500
nb_epoch = 10

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (40,40,1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=130,
          verbose=2, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 166624 samples, validate on 84 samples
Epoch 1/130
210s - loss: 0.0115 - acc: 0.9966 - val_loss: 0.2137 - val_acc: 0.9762
Epoch 2/130
207s - loss: 0.0097 - acc: 0.9972 - val_loss: 0.1400 - val_acc: 0.9643
Epoch 3/130
207s - loss: 0.0123 - acc: 0.9962 - val_loss: 0.6234 - val_acc: 0.8929
Epoch 4/130
207s - loss: 0.0106 - acc: 0.9970 - val_loss: 0.2273 - val_acc: 0.9762
Epoch 5/130
207s - loss: 0.0102 - acc: 0.9969 - val_loss: 0.4955 - val_acc: 0.8929
Epoch 6/130
206s - loss: 0.0104 - acc: 0.9970 - val_loss: 0.2571 - val_acc: 0.9524
Epoch 7/130
206s - loss: 0.0095 - acc: 0.9973 - val_loss: 0.2379 - val_acc: 0.9524
Epoch 8/130
206s - loss: 0.0102 - acc: 0.9970 - val_loss: 0.2432 - val_acc: 0.9762
Epoch 9/130
206s - loss: 0.0101 - acc: 0.9971 - val_loss: 0.3745 - val_acc: 0.9524
Epoch 10/130
206s - loss: 0.0088 - acc: 0.9975 - val_loss: 0.2574 - val_acc: 0.9405
Epoch 11/130
206s - loss: 0.0098 - acc: 0.9971 - val_loss: 0.1301 - val_acc: 0.9881
Epoch 12/130
206s - loss: 0.0083 - ac

In [25]:
model.save("4 conv with maxpool and batchnorm V2 200 epoch")

In [7]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np

In [5]:
from keras.models import load_model


model = load_model("./Models Architecture/mnist_cnn_architecture_first_sub")

In [13]:
test = np.load('test.npy')

In [14]:
import skimage.transform

test_set = []
i = 0
for elem in test:
    if i % 1000 == 0:
        print(i)
    test_set.append(skimage.transform.resize(elem, output_shape=(40,40)))
    i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000


In [15]:
test_set = np.array(test_set)

In [16]:
test_set.shape

(41428, 40, 40)

In [16]:
test_set.resize((41428,40,40,1))

In [17]:
ans = model.predict_classes(test_set, verbose=0)

In [18]:
for i in range(len(ans)):
    ans[i] = labels[ans[i]]

In [19]:
ans = pd.DataFrame(ans)

In [20]:
temp = [i+1 for i in range(len(test_set))]

temp = pd.DataFrame(temp)

In [21]:
result = pd.concat([temp, ans], axis=1)

In [22]:
result.columns = ['Id', 'Category']

In [23]:
result = result.as_matrix()

In [24]:
import numpy
numpy.savetxt("fifth_sub.csv", result, delimiter=",")